In [6]:
import numpy as np
import matplotlib.pyplot as plt

# Q1

In [181]:
def decomp1d(n, num_blocks, sub_mat_index):
    """_summary_

    Args:
        n (int): matrix lenght
        num_blocks (int): number of blocks to split into
        sub_mat_index (int): index of sub matrix row

    Returns:
        _type_: _description_
    """
    remainder = n % num_blocks
    base = n // num_blocks

    if (sub_mat_index < remainder):
        s = sub_mat_index * (base+1)
        e = s + base+1
        return s, e  # start, end
    else:
        s = (remainder * (base+1)) + (max(sub_mat_index-remainder, 0) * base)
        e = s + base
        return s, e 
   
def decomp_matrix(A, block_rows, block_cols):
    a = [[0]*block_cols for i in range(block_rows)]
    #print(a)
    m, n = A.shape
    
    for i in range(block_rows):
        s1, e1 = decomp1d(m, block_rows, i)
        
        for j in range(block_cols):
            s2, e2 = decomp1d(n, block_cols, j)
            a[i][j] = A[s1:e1, s2:e2]
    
    #print(a)
    return a

def comp_matrix(m, n, block_mat):
    B = np.zeros((m, n))
    block_rows = len(block_mat)
    block_cols = len(block_mat[0])
    
    #print(block_rows, block_cols)
    
    row_step = 0
    col_step = 0
    sub_rows = 0
    sub_cols = 0
    
    for i in range(block_rows):
        for j in range(block_cols):
            block = block_mat[i][j]
            #print(f"block.shape{block.shape}")
            sub_rows, sub_cols = block.shape
            
            for a in range(sub_rows):
                for b in range(sub_cols):
                    B[row_step + a][col_step + b] = block[a][b]
                    
            col_step += sub_cols
        row_step += sub_rows
        col_step = 0
            
    return B

def mgs(A):
    """Calculates QR decomposition of a matrix A using the modified Gram Schmidt algorithm.

    Args:
        A (numpy array): numpy matrix

    Returns:
        tuple: the pair Q, R
    """
    m, n = A.shape
    
    a = [A[:, i] for i in range(n)]
    
    Q = np.zeros((m, n))
    q = [Q[:, i] for i in range(n)]
    
    V = np.zeros((m, n))
    v = [V[:, i] for i in range(n)]
    
    R = np.zeros((n, n))
    r = [R[:, i] for i in range(n)]
    
    for i in range(n):
        v[i] = a[i]
    for i in range(n):
        r[i][i] = np.linalg.norm(v[i])
        q[i] = v[i] / r[i][i]
        for j in range(i, n):
            r[i][j] = q[i].T @ v[j]
            v[j] = v[j] - (r[i][j]*q[i])
    
    Q = np.reshape(np.array(q), (m,n))
    R = np.reshape(np.array(r), (n,n))
    
    return Q, R

In [132]:
def TSQR(A, iter):
    m, n = A.shape
    max_block_rows = 2**iter
    W = decomp_matrix(A, max_block_rows, 1)
    
    Q = 1
    R = W
    
    block_row_nums = [max_block_rows//(2**i) for i in range(iter+1)]
    for rows_num in block_row_nums:
        
        Q_temp = np.zeros((m, n))
        R_temp = []
        #print(f"Q_temp.shape={Q_temp.shape}")
        row_step = 0
        col_step = 0
        
        for i in range(rows_num):
            #print(R[i])
            Qi, Ri = np.linalg.qr(R[i][0])
            block_rows, block_cols = Qi.shape
            #print(f"Qi.shape={Qi.shape}")
            
            #print(f"block_rows={block_rows}, block_cols={block_cols}")
            for a in range(block_rows):
                for b in range(block_cols):
                    #print(row_step + a, col_step + b)
                    Q_temp[row_step + a][col_step + b] = Qi[a][b]
                    
                col_step += sub_cols
            row_step += sub_rows
            col_step = 0
                
            R_temp.append(Ri)
                    
        Q = Q @ Q_temp
        R = comp_matrix(m, n, R_temp)
        
    return Q, R


In [184]:
def TSQR(A, iter):
    m, n = A.shape
    max_block_rows = 2**iter
    W = decomp_matrix(A, max_block_rows, 1)
    
    Q = 1
    R = W
    
    #for i in range(max_block_rows):
    #    print(R[i])
    
    block_row_nums = [max_block_rows//(2**i) for i in range(iter+1)]
    for rows_num in block_row_nums:
        Q_temp = [[0]*max_block_rows for i in range(max_block_rows)]
        R_temp = [0]*max_block_rows
        
        for i in range(rows_num):
            Qi, Ri = np.linalg.qr(R[i])
            Qi, Ri = Qi[0], Ri[0]
            
            print(f"R[i]={R[i]}")
            print(" ")
            print(f"Ri={Ri}")
            print(" ")
            print(f"Qi={Qi}")
            
            
            #print(R)
            
            R_temp[i] = Ri
            Q_temp[i][i] = Qi
            
        for i in range(rows_num):
            for j in range(rows_num):
                if i!=j:
                    s, e = decomp1d(m, rows_num, i)
                    Q_temp[i][j] = np.zeros((e-s, n))
                    
        
        #for a in range(rows_num):
        #    for b in range(rows_num):
                #print(Q_temp[a][b])
        
        Q = Q @ comp_matrix(m, n, Q_temp)
        R = [comp_matrix(len(R[i*2]) + len(R[i*2 + 1]), n, 
                         [R[i*2], R[i*2 + 1]]) for i in range(rows_num//2)]
        
    return Q, R

In [185]:
A = np.random.rand(10, 4)
#A = np.eye(6)
#Q, R = np.linalg.qr(A)

#print(Q)
#print(R)

Q, R = TSQR(A, 2)
#print(Q)
for r in R:
    print(r)

R[i]=[array([[0.77034527, 0.69434286, 0.56317472, 0.78216408],
       [0.48623435, 0.54931542, 0.62225414, 0.51875395],
       [0.97557139, 0.93841213, 0.56934852, 0.47333818]])]
 
Ri=[[-1.3347641  -1.28672009 -0.96784176 -0.98635225]
 [ 0.         -0.09395835 -0.23193527 -0.03274455]
 [ 0.          0.         -0.19496684 -0.36190106]]
 
Qi=[[-0.57713964  0.51378409 -0.6347722 ]
 [-0.36428486 -0.85764362 -0.36296552]
 [-0.73089424  0.02175611  0.68214389]]
R[i]=[array([[0.63104281, 0.72247187, 0.51482162, 0.49094789],
       [0.78893946, 0.41817358, 0.24807114, 0.99449552],
       [0.01434565, 0.29256404, 0.42457242, 0.02171524]])]
 
Ri=[[-1.01036939 -0.78191338 -0.52127308 -1.08348237]
 [ 0.          0.41356975  0.46498848 -0.16990717]
 [ 0.          0.          0.1374861   0.16649394]]
 
Qi=[[-0.62456643  0.56608352 -0.53802064]
 [-0.78084261 -0.46516385  0.41702209]
 [-0.01419842  0.68056744  0.73254786]]
R[i]=[array([[0.8341296 , 0.56355992, 0.99995258, 0.91579023],
       [0.33912

IndexError: index 4 is out of bounds for axis 0 with size 4

In [112]:
num = 6
A = np.eye(num)
num_cols = 4
num_rows = 3

A_decomp = decomp_matrix(A, block_rows=num_rows, block_cols=num_cols)
"""
for i in range(num_rows):
    for j in range(num_cols):
        print(i, j)
        print(A_decomp[i][j])
"""
        
A_recomp = comp_matrix(num, num, A_decomp)
print(A_recomp)

[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]


In [113]:
my_list = [i for i in range(7)]

blocks = 4
for i in range(blocks):
    s, e = decomp1d(len(my_list), blocks, i)
    print(s, e)
    print(my_list[s:e])

0 2
[0, 1]
2 4
[2, 3]
4 6
[4, 5]
6 7
[6]
